In [20]:
import numpy as np
import pandas as pd
import io, requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.relative_locator import locate_with
import time

In [21]:
cdata = pd.read_csv('Cleaned Data')

In [22]:
cdata.shape

(30171, 10)

In [23]:
cdata.isna().sum()

Unnamed: 0                0
Unnamed: 0.1              0
name                      3
position                  0
age                       0
origin_club               0
country_origin_club     889
new_club                  0
country_new_club       5387
cost                      0
dtype: int64

In [24]:
c = 0
l = []
for u in cdata.name.isna():
    if u == True:
        l.append(c)
    c+=1

In [25]:
cdata.drop(l,axis=0,inplace=True)

In [26]:
counter = 24000

In [27]:
name = cdata.name[counter+1:counter + 2000]
ffx = webdriver.Chrome()

ffx.get("https://www.transfermarkt.com")

bigLi = []
notfound = []

for e in name:

    li = []
    li.append(e) 
    counter +=1
    try:
        # Search with player name
        ffx.find_element(By.CSS_SELECTOR, '[class="tm-header__input--search-field"]').send_keys(e) 


        # press Enter Key
        ffx.find_element(By.CSS_SELECTOR,'[class="tm-header__input--search-send"]').send_keys(Keys.ENTER) 

        
        # get player profile link like :   " https://www.transfermarkt.com/james-rodriguez/profile/spieler/88103 "
        url = ffx.find_element(By.CSS_SELECTOR,'td[class= "hauptlink"]').find_element(By.TAG_NAME,'a').get_attribute('href')
        
    except:
        ffx.find_element(By.CSS_SELECTOR, '[class="tm-header__input--search-field"]').clear()
        notfound.append(counter)
        continue

    pages = ['leistungsdatenverein', 'nationalmannschaft']

                #leistungsdatenverein --->  stat page
                #nationalmannschaft  --->  natoinal team stat page
                #marktwertverlauf   ---> markert value rank

    for page in pages:                    # switching betweem website pages from change this part  ----/profile/----
        x = url.split('/')
        x[4] = page
        for i in range(len(x)):
            x[i] = x[i] + "/"
        url = ''.join(str(u) for u in x)

        ffx.get(url)



        if page == 'leistungsdatenverein':  
            try:
                stat = ffx.find_element(By.CSS_SELECTOR, 'div[id="yw1"]').find_element(By.TAG_NAME,'tfoot')  
                
                for i, j in zip(stat.find_elements(By.CSS_SELECTOR, 'td[class="zentriert"]'), range(1,7)):
                    li.append(i.get_attribute("innerHTML"))

                for x, j in zip(stat.find_elements(By.CSS_SELECTOR, 'td[class="rechts"]'), range(7,9)):
                    li.insert(j, x.get_attribute("innerHTML"))
            except:
                while len(li) < 9 :
                    li.append('none')


        elif page == 'nationalmannschaft':  
            try:
                stat = ffx.find_element(By.CSS_SELECTOR, 'div[id="yw1"]').find_element(By.TAG_NAME,'tfoot')
                
                for i, j in zip(stat.find_elements(By.CSS_SELECTOR, 'td[class="zentriert"]'), range(9,15)):
                    li.insert(j, i.get_attribute("innerHTML"))

                for x, j in zip(stat.find_elements(By.CSS_SELECTOR, 'td[class="rechts"]'), range(15,17)):
                    li.insert(j, x.get_attribute("innerHTML"))
            except:
                while len(li) < 17 :
                    li.append('none')


    ffx.find_element(By.CSS_SELECTOR, '[class="tm-header__input--search-field"]').clear()
    bigLi.append(li)


ffx.quit()

In [39]:
counter

25999

In [40]:
notfound

[24031, 24074, 24525, 24591, 24596, 25076, 25601, 25885]

In [41]:
q = 0
for z, i  in zip(bigLi, range(len(bigLi))):
    while len(z) < 22 :
        z.append('none')
    else :
        bigLi[i] = bigLi[i][:22]
    q+=1

In [44]:
df = pd.DataFrame(bigLi, columns=['names', 'apperance', 'goals', 'assists', 'yellow_card', '2nd_ycard', 'red_card', 'drop','min', 'apperance_NT', 'goals_NT', 'assists_NT', 'yellow_card_NT', '2nd_ycard_NT', 'red_card_NT', 'drop NT','min_NT','world_rank', 'league_rank', 'club_rank', 'National_rank', 'position_rank'])                                     

In [45]:
df

,names,apperance,goals,assists,yellow_card,2nd_ycard,red_card,drop,min,apperance_NT,...,yellow_card_NT,2nd_ycard_NT,red_card_NT,drop NT,min_NT,world_rank,league_rank,club_rank,National_rank,position_rank
0,Ildian Shyti,38,3,-,4,-,-,Total :,2.284',none,...,none,none,none,none,none,none,none,none,none,none
1,Gresild Lika,63,9,4,6,-,1,Total :,4.176',none,...,none,none,none,none,none,none,none,none,none,none
2,Andrea Bianco,4,-,-,-,-,-,Total :,72',none,...,none,none,none,none,none,none,none,none,none,none
3,Amadou Sagna,49,6,10,2,-,-,Total :,3.391',9,...,-,-,-,Total :,640',none,none,none,none,none
4,George Buliga,17,-,-,3,-,-,Total :,1.145',none,...,none,none,none,none,none,none,none,none,none,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1986,Aleksey Rodionov,87,6,3,11,-,-,Total :,3.322',7,...,-,-,-,Total :,271',none,none,none,none,none
1987,Batradz Kokoev,208,5,-,33,1,1,Total :,12.304',none,...,none,none,none,none,none,none,none,none,none,none
1988,Suren Aloyan,98,-,2,-,-,158,Total :,6.911',4,...,-,-,8,Total :,360',none,none,none,none,none
1989,Vahagn Hayrapetyan,151,3,5,22,1,1,Total :,9.299',9,...,2,-,-,Total :,610',none,none,none,none,none


In [32]:
df.drop(['drop', 'drop NT'], axis= 1, inplace=True)

In [33]:
df.replace('-',0, inplace=True)

In [34]:
data = pd.read_csv('4thscraped_data')

In [35]:
data.drop(data.columns[0], axis =1 , inplace = True)

In [36]:
data = data.append(df, ignore_index = True)

In [37]:
data.to_csv('4thscraped_data')